In [16]:
import sys
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.image as mpimg
import csv

from time import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from IPython.display import display

sys.version

'3.10.4 (tags/v3.10.4:9d38120, Mar 23 2022, 23:13:41) [MSC v.1929 64 bit (AMD64)]'

# Preparing dataset


In [17]:
import pandas as pd 
def change(container):
    data = {'faces' : container}
    df = pd.DataFrame(data, columns = ['faces'])
    df.to_csv('dataset.csv', index = False)

# Image preprocessing

In [ ]:
# import numpy as np
# from PIL import Image

# img = Image.open('orig.png').convert('RGBA')
# arr = np.array(img)

# # record the original shape
# shape = arr.shape

# # make a 1-dimensional view of arr
# flat_arr = arr.ravel()

# # convert it to a matrix
# vector = np.matrix(flat_arr)

# # do something to the vector
# vector[:,::10] = 128

# # reform a numpy array of the original shape
# arr2 = np.asarray(vector).reshape(shape)

# # make a PIL image
# img2 = Image.fromarray(arr2, 'RGBA')
# img2.show()

In [20]:
container = []
original = []
faces = ['training_set/A1.jpg','training_set/A2.jpg','training_set/A3.jpg','training_set/B1.jpg','training_set/B2.jpg','training_set/B3.jpg','training_set/C1.jpg','training_set/C2.jpg','training_set/C3.jpg','training_set/D1.jpg','training_set/D2.jpg','training_set/D3.jpg','training_set/E1.jpg','training_set/E2.jpg','training_set/E3.jpg','training_set/F1.jpg','training_set/F2.jpg','training_set/F3.jpg']

for face in faces:
    face_img=Image.open(face).convert('LA')
    face_img_resize = face_img.resize((64, 64), Image.ANTIALIAS)
    new_faces = face_img_resize
    # imgplot = plt.imshow(new_faces)
    
    # convert images to numpy array
    arr_shape= np.asarray(new_faces)
    # print("The converted images in numpy array",arr_shape.shape)
    
    data = np.array(new_faces).ravel()
   
    container.append(data)
    original.append(arr_shape)
    # print('The data number of this image is ', data)
    # plt.show()



C:\Users\manny-uncharted\AppData\Local\Temp\ipykernel_13376\1242356313.py:7: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  face_img_resize = face_img.resize((64, 64), Image.ANTIALIAS)


In [33]:
# Convert 3D array to 2D list of lists
lst = []
for row in arr_shape:
    tmp = []
    for col in row:
        tmp.append(str(col))
    lst.append(tmp)

In [34]:
print(len(lst))

64


In [35]:
# 4. Save list of lists to CSV
with open('dataset.csv', 'w') as f:
    for row in lst:
        f.write(','.join(row) + '\n')

## Helper functions

In [23]:
def show_orignal_images(pixels):
    #Displaying Orignal Images
    fig, axes = plt.subplots(3, 5, figsize=(10, 5),
                             subplot_kw = {'xticks':[], 'yticks':[]})
    for i, ax in enumerate(axes.flat):
        ax.imshow(np.asarray(pixels)[i].reshape(pixels.shape), cmap='Dark2')
    plt.show()
                  

def show_eigenfaces(pca):
    #Displaying Eigenfaces
    fig, axes = plt.subplots(3, 8, figsize=(9, 4),
                             subplot_kw={'xticks':[], 'yticks':[]})
    for i, ax in enumerate(axes.flat):
        ax.imshow(pca.components_[i].reshape(64, 64), cmap='gray')
        ax.set_title("PC " + str(i+1))
    plt.show()

In [38]:
df = pd.read_csv("dataset.csv", header=None)
display(df)
# labels = df["faces"]

# show_orignal_images(pixels)

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,[ 63 255],[ 58 255],[ 54 255],[ 54 255],[ 63 255],[ 78 255],[ 83 255],[ 90 255],[ 98 255],[ 91 255],...,[ 72 255],[ 82 255],[ 84 255],[ 85 255],[ 80 255],[ 74 255],[ 70 255],[ 69 255],[ 71 255],[ 71 255]
1,[ 63 255],[ 59 255],[ 55 255],[ 54 255],[ 61 255],[ 69 255],[ 70 255],[ 74 255],[ 80 255],[ 78 255],...,[ 67 255],[ 79 255],[ 84 255],[ 85 255],[ 85 255],[ 84 255],[ 79 255],[ 75 255],[ 75 255],[ 77 255]
2,[ 61 255],[ 60 255],[ 58 255],[ 58 255],[ 59 255],[ 63 255],[ 62 255],[ 63 255],[ 72 255],[ 74 255],...,[ 66 255],[ 70 255],[ 79 255],[ 83 255],[ 85 255],[ 86 255],[ 87 255],[ 86 255],[ 86 255],[ 87 255]
3,[ 60 255],[ 60 255],[ 62 255],[ 63 255],[ 63 255],[ 67 255],[ 70 255],[ 73 255],[ 78 255],[ 79 255],...,[ 63 255],[ 64 255],[ 65 255],[ 71 255],[ 76 255],[ 78 255],[ 81 255],[ 84 255],[ 84 255],[ 81 255]
4,[ 59 255],[ 63 255],[ 66 255],[ 69 255],[ 74 255],[ 80 255],[ 87 255],[ 90 255],[ 93 255],[ 93 255],...,[ 59 255],[ 58 255],[ 59 255],[ 62 255],[ 67 255],[ 68 255],[ 75 255],[ 79 255],[ 78 255],[ 73 255]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,[190 255],[164 255],[ 99 255],[ 96 255],[ 88 255],[ 68 255],[ 36 255],[ 37 255],[ 32 255],[ 35 255],...,[ 85 255],[ 79 255],[ 72 255],[ 44 255],[ 49 255],[ 90 255],[111 255],[101 255],[ 58 255],[ 48 255]
60,[155 255],[125 255],[ 92 255],[102 255],[ 88 255],[ 81 255],[ 71 255],[ 37 255],[ 40 255],[ 36 255],...,[ 83 255],[ 79 255],[ 72 255],[ 49 255],[ 46 255],[ 61 255],[ 67 255],[ 62 255],[ 47 255],[ 44 255]
61,[124 255],[ 90 255],[ 89 255],[107 255],[ 93 255],[ 77 255],[102 255],[ 75 255],[ 34 255],[ 31 255],...,[ 83 255],[ 78 255],[ 71 255],[ 65 255],[ 61 255],[ 55 255],[ 52 255],[ 46 255],[ 41 255],[ 41 255]
62,[ 58 255],[ 54 255],[ 88 255],[112 255],[108 255],[ 96 255],[109 255],[132 255],[130 255],[ 88 255],...,[ 85 255],[ 80 255],[ 70 255],[ 71 255],[ 75 255],[ 71 255],[ 64 255],[ 56 255],[ 45 255],[ 44 255]


In [39]:
df.describe()

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
count,64,64,64,64,64,64,64,64,64,64,...,64,64,64,64,64,64,64,64,64,64
unique,37,35,34,36,33,34,32,29,28,24,...,43,42,45,46,43,42,46,38,39,38
top,[ 58 255],[ 25 255],[ 26 255],[ 23 255],[ 23 255],[ 24 255],[ 24 255],[ 27 255],[ 32 255],[ 31 255],...,[ 83 255],[ 79 255],[ 72 255],[ 74 255],[ 71 255],[ 31 255],[ 31 255],[ 29 255],[ 33 255],[ 27 255]
freq,6,4,7,7,5,7,6,8,9,9,...,5,5,6,4,5,4,5,5,5,5


In [41]:
df.isna().any().any()

False

## Split Dataset into training and testing

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(pixels, labels)

## Perform PCA

In [ ]:
pca = PCA(n_components=15).fit(x_train)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');
plt.show()